In [31]:
import os
import spacy
#
import pandas as pd
#
from spacy.matcher import Matcher
#
from textblob_de import TextBlobDE as TextBlob

In [32]:
nlp = spacy.load('de_core_news_md')
pd.set_option('display.max_colwidth', None)
os.environ["SPACY_WARNING_IGNORE"] = "W008"

In [33]:
REVIEWS_CLEANED_FILE_PATH = 'data/reviews_merged_unlabeled_cleaned.csv'

df = pd.read_csv(REVIEWS_CLEANED_FILE_PATH, sep=';')
df.head()

,Unnamed: 0,caption,rating
0,0,mega cooles ambiente konzept bürger geschmacklich lecker bürger individuelle bedürfnisse anpassen allergien vegetarisch,4.0
1,1,super sympathische mitarbeiter freundliche gemütliche atmosphäre fantastisches essen sicher öfter,5.0
2,2,nenne burger schicki-micki gerne burgerladen dortmund hüstel absoluter kult maestro tozzi gedicht gegend space burger lupo beides einfach kult,5.0
3,3,gestern dritt selben burger bestellt gestern abend magen darm burger lecker irgendetwas schmeckte komisch erwartet,1.0
4,4,fanden burger restaurant klasse individuelle möglichkeiten zudem fleischlose alternativen ambiente zeitsprung vergangenheit kartenzahlung heutzutage zeitgemäß,4.0


In [34]:
categories = ['essen', 'service', 'atmosphäre', 'preis', 'warten']

In [35]:
def spacy_most_similar(word, topn=10):
    ms = nlp.vocab.vectors.most_similar(
        nlp(word).vector.reshape(1,nlp(word).vector.shape[0]), n=topn)
    words = [nlp.vocab.strings[w] for w in ms[0][0]]
    distances = ms[2]
    return words, distances

In [36]:
spacy_most_similar(categories[0])

(['Aufessen',
  'weggefuttert',
  'drinken',
  'einzukochen',
  'koten',
  'inhaliere',
  'knabbere',
  'Bio-Essen',
  'schmatz',
  'Trinket'],
 array([[1.    , 0.8243, 0.8131, 0.7734, 0.7115, 0.7059, 0.7032, 0.6845,
         0.672 , 0.6681]], dtype=float32))

### Aspect based sentiment analysis

In [37]:
matcher = Matcher(nlp.vocab)
patterns = [
    [{'POS':'ADJ', 'OP': '+'}, {'POS':'NOUN'}],
    [{'POS':'NOUN'}, {'POS':'VERB'}, {'POS':'ADJ'}]
]
matcher.add("category", patterns)

df_matches = pd.DataFrame([], columns=['index', 'aspect', 'entity_group'])

similarity_min = 0.4

for rowIndex, wordText in enumerate(df['caption']):
    doc = nlp(wordText)
    matches = matcher(doc)
    #print(rowIndex, len(matches))

    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]
        span = doc[start:end]

        for i, s in enumerate(span):
            if s.pos_ in ['NOUN']:
                category = ''
                similarity = 0

                similarity_cat0 = nlp(categories[0]).similarity(span)
                if similarity_cat0 > similarity_min: # fits into
                    category = categories[0]
                    similarity = similarity_cat0
                    #print(rowIndex, ' - ', categories[0], ' - ', span, ' - ', similarity_cat0)

                print(span)
                similarity_cat1 = nlp(categories[1]).similarity(span)
                if similarity_cat1 > similarity_min: # fits into
                    category = categories[1]
                    similarity = similarity_cat1
                    #print(rowIndex, ' - ', categories[1], ' - ', span, ' - ', similarity_cat1)

                similarity_cat2 = nlp(categories[2]).similarity(span)
                if similarity_cat2 > similarity_min: # fits into
                    category = categories[2]
                    similarity = similarity_cat2
                    #print(rowIndex, ' - ', categories[2], ' - ', span, ' - ', similarity_cat2)

                similarity_cat3 = nlp(categories[3]).similarity(span)
                if similarity_cat3 > similarity_min: # fits into
                    category = categories[3]
                    similarity = similarity_cat3
                    #print(rowIndex, ' - ', categories[3], ' - ', span, ' - ', similarity_cat3)

                similarity_cat4 = nlp(categories[4]).similarity(span)
                if similarity_cat4 > similarity_min: # fits into
                    category = categories[4]
                    similarity = similarity_cat4
                    #print(rowIndex, ' - ', categories[4], ' - ', span, ' - ', similarity_cat4)

                #print({'index': rowIndex,
                #       'aspect': category,
                #       'entity_group': span.text,
                #       })
                df_matches = pd.concat([pd.DataFrame({'index': rowIndex,
                                                      'aspect': category,
                                                      'entity_group': span.text,
                                                      }, columns=df_matches.columns, index=[0]),
                                        df_matches])

ambiente konzept
cooles ambiente konzept
lecker bürger
individuelle bedürfnisse
sympathische mitarbeiter
gemütliche atmosphäre
freundliche gemütliche atmosphäre
fantastisches essen
burger schicki-micki
absoluter kult
burger lupo
selben burger
individuelle möglichkeiten
ambiente zeitsprung
alternativen ambiente zeitsprung
fleischlose alternativen ambiente zeitsprung
bester service
lecker burger
fairen preis
ambiente kartenzahlung
coolen ambiente kartenzahlung
leckere burger
freundliche bedienung
klasse auswahl
milchshake kombination
sterne milchshake kombination
veganer vegetarier
bescheidenen services
kreative auswahl
schlechten erfahrungen
tolle atmosphäre
leckere burger
leckere burger
nette bedienung
burger d-dorf
handgemachte burger
lecker gast
tische minuten
kalter burger
nettes personal
erstklassige burger
witziges ambiente
lieblingslokale stadt
eingerichtetes restaurant
lockeren servicemitarbeitern
größer sparmenüs
versteckte kamera
cooles ambiente
fairen preisen
bester burger
ba

/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_56385/384582423.py:26: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat0 = nlp(categories[0]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_56385/384582423.py:33: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat1 = nlp(categories[1]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_56385/384582423.py:39: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat2 = nlp(categories[2]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_56385/384582423.py:45: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat3 = nlp(categories[3]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_56385/384582423.py:51: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat

beste döner
wurden.die geschwindigkeit
freundliche bedienung
saucen ordnung
echte dönner
dönerläden stadt
schlechte qualität
schlechteste döner
peperoni döner
beste peperoni döner
döner preis
einziger stern
döner schmeckt absoluter
absoluter standard
appetit gesehen anderer
anderer zutaten
negative emotionen
qualität verschlechtert rote
soße geschmack
rote soße geschmack
restaurant gerannt letztes
letztes döner
negativen sinne
unfreundliche bedienung
soße ertränkt
monatlichen preiserhöhungen
look kind
erwartete döner
teurer preis
besseren döner
unverhältnismäßiger preisanstieg
döner plastik
durchschnittlicher standard
günstigste preis
besonderes fett
leidenschaftlicher essenkritiker
totzdessen gemüse
döner totzdessen gemüse
soße brot
richtiges fleisch
schlechte qualität
döner stadt
beste döner stadt
soße soße
ähnlichem sterne
netten mitarbeiter
letzter gefühl
düsseldorfer innenstadt
bessere gegessen
update chance
lustiges personal
herzliches lustiges personal
lieblingsdöner herzliches 

/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_56385/384582423.py:26: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat0 = nlp(categories[0]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_56385/384582423.py:33: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat1 = nlp(categories[1]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_56385/384582423.py:39: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat2 = nlp(categories[2]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_56385/384582423.py:45: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat3 = nlp(categories[3]).similarity(span)
/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_56385/384582423.py:51: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat

pizza essen getränke
getränke theke
pizza niveau
zuvorkommendes personal
freundliches zuvorkommendes personal
schönen abend
neapolitanische pizza
vernünftige neapolitanische pizza
beste pizza
italiens stimmung
geschmackssache pizza
nettes personal
angenehmes ambiente
neapolitanische pizza
hervorragende neapolitanische pizza
fruendliche atmosphäre
schöne auswahl
leckere pizza
jüngere volk


In [38]:
df_matches

,index,aspect,entity_group
0,332,,jüngere volk
0,330,essen,leckere pizza
0,329,,schöne auswahl
0,329,atmosphäre,fruendliche atmosphäre
0,328,,hervorragende neapolitanische pizza
...,...,...,...
0,1,atmosphäre,sympathische mitarbeiter
0,0,,individuelle bedürfnisse
0,0,essen,lecker bürger
0,0,atmosphäre,cooles ambiente konzept


#### TextBlob sentiment analysis

In [55]:
for i, row in df_matches.iterrows():
    print(row['aspect'], row['entity_group'])
    sentiment = TextBlob(row['entity_group']).sentiment
    #df_matches['sentiment'] = sentiment
    print(row['index'], row['aspect'], row['entity_group'], sentiment.polarity, sentiment.subjectivity)

#df_matches['sentiment'] = sentiment
#print(df_matches)

 jüngere volk
332  jüngere volk 0.0 0.0
essen leckere pizza
330 essen leckere pizza 0.0 0.0
 schöne auswahl
329  schöne auswahl 1.0 0.0
atmosphäre fruendliche atmosphäre
329 atmosphäre fruendliche atmosphäre 0.0 0.0
 hervorragende neapolitanische pizza
328  hervorragende neapolitanische pizza 1.0 0.0
essen neapolitanische pizza
328 essen neapolitanische pizza 0.0 0.0
atmosphäre angenehmes ambiente
327 atmosphäre angenehmes ambiente 1.0 0.0
service nettes personal
327 service nettes personal 1.0 0.0
essen geschmackssache pizza
326 essen geschmackssache pizza 0.0 0.0
atmosphäre italiens stimmung
326 atmosphäre italiens stimmung 0.0 0.0
 beste pizza
324  beste pizza 0.0 0.0
essen vernünftige neapolitanische pizza
323 essen vernünftige neapolitanische pizza 0.0 0.0
essen neapolitanische pizza
323 essen neapolitanische pizza 0.0 0.0
essen schönen abend
322 essen schönen abend 0.0 0.0
atmosphäre freundliches zuvorkommendes personal
321 atmosphäre freundliches zuvorkommendes personal 1.0 0.0


### Updating models

In [40]:
nlp("essen").similarity(nlp("pizza"))

0.4386588382204547

In [41]:
nlp('pizz').vector_norm

0